# INDICE INVERTIDO / INDICE INVERSO

## OBTENCIÓN DE TODOS LOS TERMINOS
Leemos el archivo llamado DocumentoPreprocesado.txt para poder obtener todos los terminos del documento (sin repetirlos)

In [2]:
terminos=[]
tabla=[]
documentos=[]
with open("DocumentoPreprocesado.txt",encoding="utf8") as file1:#LEECTURA DEL DOCUMENTOS PARA LA OBTENCION DE TERMINOS  
    for linea in file1:
        documentos.append(linea)
        words = linea.split() 
        for word in words:
            terminos.append(word)                               #AGREGAMOS LA PALABRA A LA LISTA TERMINOS 
    terminos = list(set(terminos))                              #PARA QUITAR LOS TERMINOS REPETIDOS DE LA LISTA
file1.close()

#### IMPRIMIMOS LOS TERMINOS

In [3]:
import pandas as pd
pd.DataFrame(terminos)

,0
0,sergej
1,smirkingface
2,cafe
3,seriedemods
4,parte
...,...
1219,defenderte
1220,dora
1221,p
1222,locomotora


In [4]:
with open("DocumentoPreprocesado.txt",encoding="utf8") as file1:  
    for linea in file1:
        words = linea.split()                      #WORDS SON LAS PALABRAS DE NUESTRO DOCUMENTO/LINEA (LISTA)
        numDePalabras = dict.fromkeys(terminos, 0) #ASIGNAMOS 0 A LOS TERMINOS DE NUESTRO DICCIONARIO (NUMDEPALABRAS)
        for word in words:                         #POR CADA PALABRA DE NUESTRO DOCUMENTO
            numDePalabras[word] += 1               #CONTAMOS CUANTAS VECES SE ENCUENTRA ESA PALABRA EN EL DOCUMENTO Y ACTUALIZAMOS EL DICCIONARIO
        tabla.append(numDePalabras)                #NUESTRA TABLA ES UNA LISTA DE DICCIONARIOS (1 DICCIONARIO POR CADA DOCUMENTO)
        documentos.append(words)
file1.close()

## INDICE INVERSO CON OCURRENCIA

In [5]:
indice_invertido = {} 
i=0
for documento in documentos:                   #PARA CADA UNO DE LOS DOCUMENTOS
    for termino in terminos:                   #PARA CADA UNO DE LOS TERMINOS
        ubi_fre= []                            #LISTA QUE CONTIENE LA UBICACION DEL TERMINO (EL DOCUMENTO) Y SU FRECUENCIA
        if termino in documento:               #SI EL TERMINO SE ENCONTRO EN EL DOCUMENTO
            if termino not in indice_invertido:#SI AUN NO ESTA EL TERMINO EN LA LISTA INVERTIDA
                indice_invertido[termino] = [] #SE AGREGA AL DICCIONARIO EL TERMINO Y UNA LISTA VACIA DONDE SE ALMACENARÁ
                                               #UBI_FRE
            if termino in indice_invertido:    #SI EL TERMINO ESTÁ EN EL INDICE INVERTIDO
                frecuencia=tabla[i][termino]
                ubi_fre.append(i+1)            #SE AGREGA A UBI_FRE EL NUMERO DEL DOCUMENTO 
                ubi_fre.append(frecuencia)#SE AGREGA A UBI_FRE LA FRECUENCIA DEL TERMINO EN ESE DOCUMENTO
                if frecuencia != 0:
                    indice_invertido[termino].append(ubi_fre)#SE AGREGA AL INDICE INVERTIDO UBI_FRE
    i=i+1
    if i == len(tabla):
        break

In [6]:
import pandas as pd   
df = pd.DataFrame([[key, indice_invertido[key]] for key in indice_invertido.keys()], columns=['Vocabulario ', 'Ocurrencias índice invertido'])#SE IMPRIMRE EL DATAFRAME CREADO CON EL NUEVO DICCIONARIO CREADO POR LA FUNCION IDF
df     #IMPRIMIMOS EL DATA FRAME

,Vocabulario,Ocurrencias índice invertido
0,v,"[[286, 1]]"
1,tweeted,"[[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1..."
2,l,"[[80, 1], [242, 1], [247, 1], [333, 1]]"
3,im,"[[1, 1], [225, 1]]"
4,dia,"[[10, 1], [63, 1], [85, 1], [157, 1], [195, 2]..."
...,...,...
1219,demasiado,"[[349, 1]]"
1220,suspender,"[[350, 1]]"
1221,presente,"[[350, 1]]"
1222,momentaneamente,"[[350, 1]]"


### TF

In [7]:
import math
def TF(documento):                              #RECIBIMOS UN DICCIONARIO QUE ES NUESTRO DOCUMENTO CON LA FRECUENCIA
    tfDict = {}                                 #CREAMOS UN DICCIONARIO PARA TF
    for word, fij in documento.items():         #SE EMPIEZA CON LA LECTURA DE CADA PALABRA DEL DOCUMENTO Y OBTENEMOS SU FRECUENCIA
        if fij != 0:                           
            tfDict[word] = 1 + math.log2(fij)   #SI 𝑓𝑖,𝑗 > 0 SE REALIZA LA OPERACIÓN
        else:
            tfDict[word] = 0                    #EN OTRO CASO SE ASIGNA 0
    return tfDict 

tf=[]                        #LISTA DE DICCIONARIOS DE TF
for documento in tabla:      #SE RECORREN LOS DICCIONARIOS DE LA LISTA TABLA
    tf.append(TF(documento)) #SE AGREGA A LA LISTA NUEVOS DICCIONARIOS CREADOS POR LA FUNCIÓN TF

## IDF

In [8]:
def IDF(documentos):                                #RECIBIMOS TODOS LOS DOCUMENTOS
    import math                                     #IMPORTAMOS MATH PARA PODER TRABAJAR CON LOG BASE 2
    idfDict = {}                                    #CREAMOS UN NUEVO DICCIONARIO PARA IDF
    N = len(documentos)                             #CON LEN OBTENEMOS LA CANTIDAD DE DOCUMENTOS Y SE ASIGNA A N
    idfDict = dict.fromkeys(documentos[0].keys(), 0)#ASIGNAMOS A LOS CONTENIDOS DE LOS DICCIONARIOS 0
    for doc in documentos:                          #VOLVEMOS A OBTENER LA FRENCUENCIA DE LOS TERMINOS PARA ESTE DICCIONARIO
        for word, ni in doc.items():
            if ni > 0:                              #SOLO AQUELLOS QUE SU VALOR SEA MAYOR A 0 PARA PODER REALIZAR LA DIVISIÓN
                idfDict[word] += 1
    
    for word, ni in idfDict.items():                #RECORREMOS EL DICCIONARIO CREADO ANTERIORMENTE
        idfDict[word] = math.log2(N / float(ni))    #ASIGNAMOS LA OPERACIÓN A LA PALABRA DEL DICCIONARIO IDFDICT 
    return idfDict    

idfs = IDF(tabla) 

## TF IDF

In [9]:
def TFIDF(documentotf, idfs):                 #FUNCION TFIDF RECIBE TANTO LA LISTA TF COMO IDF
    tfidf = {}                                #CREAMOS DICCIONARIO TFIDF
    for word, valor in documentotf.items():   #CADA PALABRA DEL DOCUMENTO (TF)
        tfidf[word] = valor*idfs[word]        #EL VALOR DE LA PALABRA DEL DOCUMENTO SE MULTIPLICA POR EL VALOR DE LA FRECUENCIA INVERSA DE LA PALABRA 
    return tfidf  
tfidf=[]                                  #LISTA TFIDF PARA MOSTRAR NUESTRA TABLA 
for documentotf in tf:                    #VAMOS RECORRIENDO LA LISTA DE DICCIONARIOS (CADA DICCIONARIO ES UN DOCUMENTO)
    tfidf.append(TFIDF(documentotf, idfs))#AGREMOS EL DICCIONARIO RETORNADO A NUESTRA LISTA TFIDF

### INDICE INVERTIDO TF IDF

In [10]:
indice_invertido_tfidf = {} 
conjunto = ()
i=0
for documento in documentos:                   #PARA CADA UNO DE LOS DOCUMENTOS
    for termino in terminos:                   #PARA CADA UNO DE LOS TERMINOS
        ubi_idf= []                            #LISTA QUE CONTIENE LA UBICACION DEL TERMINO (EL DOCUMENTO) Y SU FRECUENCIA
        if termino in documento:               #SI EL TERMINO SE ENCONTRO EN EL DOCUMENTO
            if termino not in indice_invertido_tfidf:#SI AUN NO ESTA EL TERMINO EN LA LISTA INVERTIDA
                indice_invertido_tfidf[termino] = [] #SE AGREGA AL DICCIONARIO EL TERMINO Y UNA LISTA VACIA DONDE SE ALMACENARÁ
                                               #UBI_FRE
            if termino in indice_invertido_tfidf:    #SI EL TERMINO ESTÁ EN EL INDICE INVERTIDO
                peso=tfidf[i][termino]
                ubi_idf.append(i+1)            #SE AGREGA A UBI_FRE EL NUMERO DEL DOCUMENTO 
                ubi_idf.append(peso)#SE AGREGA A UBI_FRE LA FRECUENCIA DEL TERMINO EN ESE DOCUMENTO
                if peso != 0:
                    indice_invertido_tfidf[termino].append(ubi_idf)#SE AGREGA AL INDICE INVERTIDO UBI_FRE
    i=i+1
    if i == len(tabla):
        break

In [11]:
import pandas as pd   
df = pd.DataFrame([[key, indice_invertido_tfidf[key]] for key in indice_invertido_tfidf.keys()], columns=['Vocabulario ', 'Peso TF IDF índice invertido'])#SE IMPRIMRE EL DATAFRAME CREADO CON EL NUEVO DICCIONARIO CREADO POR LA FUNCION IDF
df.to_excel('TFIDF.xlsx')
df   

,Vocabulario,Peso TF IDF índice invertido
0,v,"[[286, 8.451211111832329]]"
1,tweeted,[]
2,l,"[[80, 6.451211111832329], [242, 6.451211111832..."
3,im,"[[1, 7.451211111832329], [225, 7.4512111118323..."
4,dia,"[[10, 4.750771393691236], [63, 4.7507713936912..."
...,...,...
1219,demasiado,"[[349, 8.451211111832329]]"
1220,suspender,"[[350, 8.451211111832329]]"
1221,presente,"[[350, 8.451211111832329]]"
1222,momentaneamente,"[[350, 8.451211111832329]]"


## CONSULTA

In [12]:
def elimina_tildes(busqueda):
    s = ''.join((c for c in unicodedata.normalize('NFD',busqueda) if unicodedata.category(c) != 'Mn'))
    return s

In [13]:
stop_words_sp_en = set()
import io  
import unicodedata #PARA LA FUNCION DE ELIMINAR TILDES
import spacy #PARA LA FUNCION DE LEMATIZACION
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
stop_words = set(stopwords.words('Spanish')) 
#LE QUITAMOS LAS TILDES DEBIDO A QUE AL BUSCAR LE HABREMOS QUITADOS LAS TILDES A NUESTROS DOCUMENTOS
for palabra in stop_words:
    palabrasinT=elimina_tildes(palabra)
    stop_words_sp_en.add(palabrasinT)
#MEZCLAMOS LAS STOPWORDS EN INGLES Y ESPAÑOL
stop_words_sp_en.update(set(stopwords.words('English'))) 

In [14]:
def lematizacion(busqueda):
    sp = spacy.load('es_core_news_sm')#LEMATIZA EN ESPAÑOL (SE DESCARGÓ LIBRERY)
    #sp = spacy.load('en_core_web_sm')
    consulta = ''
    query=sp(busqueda)
    for words in query:
        consulta = consulta + words.lemma_ + " "
    return consulta

In [15]:
def elimina_PV(busqueda):
    palabra_sinPV=''
    words = busqueda.split() 
    for word in words:
        if not word in stop_words_sp_en:
            palabra_sinPV = palabra_sinPV+word+" "
    return palabra_sinPV

In [16]:
def preprocesamiento_Consulta(busqueda):
    #query= lematizacion(elimina_PV(elimina_tildes(busqueda).lower()))
    query= elimina_PV(elimina_tildes(lematizacion(busqueda).lower()))
    return query

#Frases de prueba

juegos del hambre
partido aburrido
un novio divertido
venta de vestidos largos
mejores canciones de juan gabriel

In [19]:

while(True):
    Query=''
    print("==============================================================================================")
    print("Realiza tu consulta Consulta")
    lectura = input()
    Query = preprocesamiento_Consulta(lectura)
    aparicion(Query.rstrip())
    print("==============================================================================================")
    print("¿Deseas hacer otra consulta?")
    respuesta = input()
    if respuesta == 'no':
        print("==============================================================================================")
        print("Gracias por tu consulta")
        print("==============================================================================================")
        break

Realiza tu consulta Consulta
juegos del hambre
Numero de documentos obtenidos para el query y su ocurrencia
Mas Relevantes
Documento:  28
Menos Relevantes
Documento:  15
Documento:  350
Documento:  21
Documento:  8
¿Deseas hacer otra consulta?
si
Realiza tu consulta Consulta
partido aburrido
Numero de documentos obtenidos para el query y su ocurrencia
Mas Relevantes
Documento:  341
Menos Relevantes
Documento:  292
Documento:  275
Documento:  159
¿Deseas hacer otra consulta?
si
Realiza tu consulta Consulta
un novio divertido
Numero de documentos obtenidos para el query y su ocurrencia
Mas Relevantes
Documento:  50
Menos Relevantes
Documento:  138
Documento:  63
Documento:  187
Documento:  43
¿Deseas hacer otra consulta?
si
Realiza tu consulta Consulta
venta de vestidos largos
Numero de documentos obtenidos para el query y su ocurrencia
Mas Relevantes
Documento:  188
Menos Relevantes
¿Deseas hacer otra consulta?
si
Realiza tu consulta Consulta
mejores canciones de juan gabriel
Numero de 

In [18]:
import pprint
import operator
def aparicion(query):
    documentos_Presentes= []
    dict_aparicion ={}
    print("==============================================================================================")
    palabras = query.split(" ")
    #IMPRIME LOS DOCUMENTOS EN LOS QUE APARECE LAS PALABRAS JUNTO CON SU PESO
    #print("Palabra y sus pesos")
    for palabra in palabras:
        if palabra in indice_invertido_tfidf:
            contenido = indice_invertido_tfidf[palabra]
            for cont in contenido:
                documentos_Presentes.append(cont[0])                #SE TOMA EL [0] DEBIDO A QUE ESTE ES EL DOCUMENTO
            if len(contenido) == 0:                                 #SI LA LISTA ESTA VACIA SIGNIFICA QUE APARECEN EN TODOS LOS DOCUMENTOS
                print("=>",palabra," esta en todos los documentos")
            else:
                '''print("=>",palabra,": ")
                pprint.pprint(contenido)'''
    #PARA OBTENER OCURRENCIA DE DOCUMENTOS (NO SE HACE USO DE LOS PESOS)
    for doc in documentos_Presentes:
        if doc in dict_aparicion:
            dict_aparicion[doc]+=1
        else:
            dict_aparicion[doc]=1
    #print(dict_aparicion)
    #IMPRIMIR DOCUMENTOS ORDENADOS POR EL MAYOR RELEVANTE
    print("Numero de documentos obtenidos para el query y su ocurrencia")
    ordenados = list(reversed(sorted(dict_aparicion.items(), key=operator.itemgetter(1))))
    for i in range(len(ordenados)):
        if i == 0:
            print("Mas Relevantes")
            print("Documento: ",ordenados[i][0])
            print("Menos Relevantes")
        else:
            print("Documento: ",ordenados[i][0])